In [1]:
#test loss and metrics

In [2]:
import os
import json
import time
import copy
from copy import deepcopy
from collections import defaultdict

import numpy as np
import math
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models

from skimage import io

import matplotlib.pyplot as plt
from matplotlib import patches, patheffects

import imgaug as ia
from imgaug import augmenters as iaa

from sklearn.model_selection import train_test_split

from tqdm import tqdm
from pprint import pprint

In [14]:
DEVICE = torch.device("cpu")

In [17]:
def k_hot_embedding(labels, num_classes):
    khot = torch.eye(num_classes)[labels.data.cpu()]
    khot = khot.sum(1).clamp(0,1)
    return khot

def prep_yTrue(y_true):
    #prep y_true
    y_true_khot = k_hot_embedding(y_true, num_classes = NUM_CLASSES + 1)
    y_true_khot = y_true_khot[:, :-1] #last element is dummy
    y_true_khot = y_true_khot.to(DEVICE)
    return y_true_khot

def criterion(y_pred, y_true):
    y_true_khot = prep_yTrue(y_true)
    
    #calculate loss
    loss = F.binary_cross_entropy_with_logits(y_pred, y_true_khot)
    return loss

In [39]:
num_class = 4
NUM_CLASSES = num_class
max_tags = 3
batch_size = 5
THRESHOLD=0.5

In [12]:
y_true = np.array([[0, 2, 3],
       [0, 1, 2],
       [3, 4, 0],
       [1, 2, 4],
       [2, 3, 0]])
y_true.sort(axis=1)

In [15]:
y_true = torch.from_numpy(y_true)

In [16]:
y_true

tensor([[0, 2, 3],
        [0, 1, 2],
        [0, 3, 4],
        [1, 2, 4],
        [0, 2, 3]])

In [18]:
y_true_khot = k_hot_embedding(y_true, num_class+1)

In [19]:
y_true_khot

tensor([[1., 0., 1., 1., 0.],
        [1., 1., 1., 0., 0.],
        [1., 0., 0., 1., 1.],
        [0., 1., 1., 0., 1.],
        [1., 0., 1., 1., 0.]])

In [21]:
y_true_khot = y_true_khot[:, :-1] #last element is dummy

In [22]:
y_true_khot

tensor([[1., 0., 1., 1.],
        [1., 1., 1., 0.],
        [1., 0., 0., 1.],
        [0., 1., 1., 0.],
        [1., 0., 1., 1.]])

In [23]:
F.binary_cross_entropy_with_logits(y_true_khot, y_true_khot)

tensor(0.4462)

In [31]:
F.binary_cross_entropy(torch.sigmoid(y_true_khot), y_true_khot)

tensor(0.4462)

In [33]:
torch.sigmoid(y_true_khot)

tensor([[0.7311, 0.5000, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.5000],
        [0.7311, 0.5000, 0.5000, 0.7311],
        [0.5000, 0.7311, 0.7311, 0.5000],
        [0.7311, 0.5000, 0.7311, 0.7311]])

In [34]:
#test stats

In [37]:
y_true_khot = prep_yTrue(y_true).byte()

In [38]:
y_true_khot

tensor([[1, 0, 1, 1],
        [1, 1, 1, 0],
        [1, 0, 0, 1],
        [0, 1, 1, 0],
        [1, 0, 1, 1]], dtype=torch.uint8)

In [44]:
y_pred_sgmd = torch.sigmoid(y_true_khot.float()) > THRESHOLD

In [45]:
y_pred_sgmd

tensor([[1, 0, 1, 1],
        [1, 1, 1, 0],
        [1, 0, 0, 1],
        [0, 1, 1, 0],
        [1, 0, 1, 1]], dtype=torch.uint8)

In [46]:
batch_true_pos = (y_pred_sgmd * y_true_khot).sum(0)

In [48]:
batch_false_pos = y_pred_sgmd.sum(0) - batch_true_pos

In [50]:
batch_false_neg = y_true_khot.sum(0) - batch_true_pos

In [77]:
batch_false_neg

tensor([0, 0, 0, 0])

In [52]:
def calc_epoch_stats(running_true_pos, running_false_pos, running_false_neg):
    #per class stats
    percision_per_class = np.nan_to_num(running_true_pos/(running_true_pos+running_false_pos),0)
    recall_per_class= np.nan_to_num(running_true_pos/(running_true_pos+running_false_neg),0)
    f1_per_class = np.nan_to_num((2 * (percision_per_class * recall_per_class)/(percision_per_class + recall_per_class)),0)
    
    #calc macro scores
    percision_macro = np.mean(percision_per_class)
    recall_macro = np.mean(recall_per_class)
    f1_macro = np.mean(f1_per_class)
    
    return percision_macro, recall_macro, f1_macro

In [82]:
true_pos = np.array([1,0,2,2])
false_pos = np.array([1,3,1,0])
false_neg = np.array([3,1,0,0])

In [83]:
percision_macro, recall_macro, f1_macro = calc_epoch_stats(true_pos, false_pos, false_neg)

/home/spacor/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [86]:
percision_macro

0.5416666666666666

In [102]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [87]:
y_pred = np.array([
    [0,0,0,0],
    [1,0,1,0],
    [0,1,1,1],
    [0,1,1,1],
    [1,1,0,0],
])
y_true = np.array([
    [1,0,0,0],
    [0,1,1,0],
    [1,0,1,1],
    [1,0,0,1],
    [1,0,0,0],
])

In [88]:
f1_score(y_true, y_pred, average='macro')

0.5333333333333333

In [89]:
precision_score(y_true, y_pred, average='macro')

0.5416666666666666

In [90]:
recall_score(y_true, y_pred, average='macro')

0.5625

In [107]:
np.concatenate([y_pred, y_true])

array([[0, 0, 0, 0],
       [1, 0, 1, 0],
       [0, 1, 1, 1],
       [0, 1, 1, 1],
       [1, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 1, 0],
       [1, 0, 1, 1],
       [1, 0, 0, 1],
       [1, 0, 0, 0]])

In [108]:
#f1 loss
y_pred = torch.from_numpy(y_pred)
y_true = torch.from_numpy(y_true)

In [110]:
(y_true * y_pred

tensor([[0, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 1],
        [0, 0, 0, 1],
        [1, 0, 0, 0]])

In [150]:
tp = (y_true * y_pred).sum(0).float()
# tn = ((1-y_true) * (1-y_pred)).sum(0).float()
fp = ((1-y_true) * y_pred).sum(0).float()
fn = (y_true * (1-y_pred)).sum(0).float()

p = tp / (tp + fp)
r = tp / (tp + fn)

f1 = 2*p*r / (p+r)
f1[torch.isnan(f1)] = 0
f1_loss = 1-f1.mean()